In [1]:
import pandas as pd
import os
import shutil
import numpy as np
import glob
import subprocess

from QC.utils import shell_do

In [3]:
# report = pd.read_csv('/data/CARD/PD/ibx_data/P1/F4SEttehadieh_P1_Genotyping_Report.txt', sep='\s+')

/usr/local/Anaconda/envs/py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
basedir = f'/data/CARD/PD/ibx_data'
p1_idat_dir = f'{basedir}/P1/Raw_Data'
p23_idat_dir = f'{basedir}/P2-3/Raw_Data'
ped_dir = f'{basedir}/ped'
bed_dir = f'{basedir}/bed'
gtc_dir = f'{basedir}/gtc'
ref_dir = '../ref'
fasta = f'{ref_dir}/human_g1k_v37.fasta'
swarm_scripts_dir = f'{basedir}/swarm_scripts'

os.makedirs(ped_dir, exist_ok=True)
os.makedirs(bed_dir, exist_ok=True)
os.makedirs(gtc_dir, exist_ok=True)
os.makedirs(swarm_scripts_dir, exist_ok=True)

ilmn_files_path = '/data/CARD/PD/GP2/ilmn_files'
bpm = f'{ilmn_files_path}/NeuroBooster_20042459_A1.bpm'
egt = f'{ilmn_files_path}/NBSCluster_file_n1393_011921.egt'
iaap = f'{ilmn_files_path}/iaap-cli/iaap-cli'

In [5]:
with open(f'{swarm_scripts_dir}/idat_to_ped.swarm', 'w') as f:
    for idat_dir in [p1_idat_dir,p23_idat_dir]:
        idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{ped_dir}/ \
-f {idat_dir}/ \
-p \
-t 16'
        
        f.write(f'{idat_to_ped_cmd}\n')
f.close()

!cat {swarm_scripts_dir}/idat_to_ped.swarm

/data/CARD/PD/GP2/ilmn_files/iaap-cli/iaap-cli gencall /data/CARD/PD/GP2/ilmn_files/NeuroBooster_20042459_A1.bpm /data/CARD/PD/GP2/ilmn_files/NBSCluster_file_n1393_011921.egt /data/CARD/PD/ibx_data/ped/ -f /data/CARD/PD/ibx_data/P1/Raw_Data/ -p -t 16
/data/CARD/PD/GP2/ilmn_files/iaap-cli/iaap-cli gencall /data/CARD/PD/GP2/ilmn_files/NeuroBooster_20042459_A1.bpm /data/CARD/PD/GP2/ilmn_files/NBSCluster_file_n1393_011921.egt /data/CARD/PD/ibx_data/ped/ -f /data/CARD/PD/ibx_data/P2-3/Raw_Data/ -p -t 16


In [7]:
!swarm -f {swarm_scripts_dir}/idat_to_ped.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

18181232


In [6]:
with open(f'{swarm_scripts_dir}/idat_to_gtc.swarm', 'w') as f:
    for idat_dir in [p1_idat_dir,p23_idat_dir]:
        idat_to_gtc_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{gtc_dir}/ \
-f {idat_dir}/ \
-g \
-t 16'
        
        f.write(f'{idat_to_gtc_cmd}\n')
f.close()

!cat {swarm_scripts_dir}/idat_to_gtc.swarm

/data/CARD/PD/GP2/ilmn_files/iaap-cli/iaap-cli gencall /data/CARD/PD/GP2/ilmn_files/NeuroBooster_20042459_A1.bpm /data/CARD/PD/GP2/ilmn_files/NBSCluster_file_n1393_011921.egt /data/CARD/PD/ibx_data/gtc/ -f /data/CARD/PD/ibx_data/P1/Raw_Data/ -g -t 16
/data/CARD/PD/GP2/ilmn_files/iaap-cli/iaap-cli gencall /data/CARD/PD/GP2/ilmn_files/NeuroBooster_20042459_A1.bpm /data/CARD/PD/GP2/ilmn_files/NBSCluster_file_n1393_011921.egt /data/CARD/PD/ibx_data/gtc/ -f /data/CARD/PD/ibx_data/P2-3/Raw_Data/ -g -t 16


In [8]:
!swarm -f {swarm_scripts_dir}/idat_to_gtc.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

18181236


In [9]:
ped_filenames = [p.split('.')[0] for p in glob.glob(f'{basedir}/ped/*.ped')]
map_file = f'{ped_dir}/NeuroBooster_20042459_A1.map'
for filename in ped_filenames:
    ped = f'{filename}.ped'
    out_map = f'{filename}.map'
    if os.path.isfile(ped):
        shutil.copyfile(src=map_file, dst=out_map)
    else:
        print(f'{ped} does not exist!')
        print(f'{out_map} creation cancelled')

In [12]:
with open(f'{swarm_scripts_dir}/make_bed.swarm', 'w') as f:
    for filename in ped_filenames:
        ped = f'{filename}'
        make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {filename}'

        f.write(f'{make_bed_cmd}\n')
f.close()


In [13]:
!swarm -f {swarm_scripts_dir}/make_bed.swarm -g 32 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

18185588


In [14]:
with open(f"{bed_dir}/merge_bed.list", 'w') as f:
    for filename in ped_filenames:
        bed = f'{filename}'
        f.write(f'{bed}\n')
f.close()

with open(f"{swarm_scripts_dir}/merge.swarm", 'w') as f:

    plink_merge_cmd = f'\
plink \
--merge-list {bed_dir}/merge_bed.list \
--make-bed \
--out {bed_dir}/ibx'
    f.write(f"{plink_merge_cmd}")
f.close()

In [15]:
!swarm -f {swarm_scripts_dir}/merge.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

18187465


In [3]:
gtc_list = [s.split('.')[0] for s in glob.glob(f'{gtc_dir}/*.gtc')]

with open(f'{swarm_scripts_dir}/gtc2vcf.swarm','w') as f:
    for gtc in gtc_list:
        gtc2vcf_cmd = f'/data/vitaled2/GenoTools/GTCtoVCF/gtc_to_vcf.py --gtc-paths {gtc}.gtc --manifest-file {bpm} --genome-fasta-file {fasta} --include-attributes {{GT,GQ,BAF,LRR}} --skip-indels --output-vcf-path {gtc}.vcf'
        f.write(f'{gtc2vcf_cmd}\n')
f.close()

In [17]:
!swarm -f {swarm_scripts_dir}/gtc2vcf.swarm -g 64 -t 32 --time=10:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

18187469


In [18]:
for gtc in gtc_list:
    vcf_view_cmd = f'bcftools view -Oz -o {gtc}.vcf.gz {gtc}.vcf'
    shell_do(vcf_view_cmd)

Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204835450113_R08C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204835450113_R08C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204825020104_R08C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204825020104_R08C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204835450153_R04C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204835450153_R04C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204835450051_R02C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204835450051_R02C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204835450114_R03C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204835450114_R03C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204842400028_R06C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204842400028_R06C01.vcf
Executing: bcftools view -Oz -o /data/CARD/PD/ibx_data/gtc/204842400095_R01C01.vcf.gz /data/CARD/PD/ibx_data/gtc/204842400095_R01C01.vcf
Executing: bcftools view -Oz -o /data/CAR

In [19]:
for gtc in gtc_list:
    vcf_tabix_cmd = f'tabix -p vcf {gtc}.vcf.gz'
    shell_do(vcf_tabix_cmd)

Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204835450113_R08C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204825020104_R08C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204835450153_R04C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204835450051_R02C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204835450114_R03C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400028_R06C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400095_R01C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400119_R03C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204835450122_R05C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204835450123_R08C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204835450114_R01C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400014_R08C01.vcf.gz
Executing: tabix -p vcf /data/CARD/PD/ibx_data/gtc/204842400096_

In [20]:
# !bcftools merge --merge all {gtc_dir}/*.vcf.gz -Ov -o {gtc_dir}/ibx.vcf
bcftools_merge_cmd = f'bcftools merge --merge all {gtc_dir}/*.vcf.gz -Ov -o {gtc_dir}/ibx.vcf'
subprocess.run(bcftools_merge_cmd, stdout=subprocess.PIPE, shell=True)

CompletedProcess(args='bcftools merge --merge all /data/CARD/PD/ibx_data/gtc/*.vcf.gz -Ov -o /data/CARD/PD/ibx_data/gtc/ibx.vcf', returncode=0, stdout=b'')

In [ ]:
# line_count = 0
# with open(f'{gtc_dir}/ibx.vcf', "r") as ifile:
#     for line in ifile:
#         line_count += 1
#         if line.startswith("#CHROM"):
#             break


# vcf = pd.read_csv(f'{gtc_dir}/ibx.vcf', sep='\s+', skiprows=line_count-1, low_memory=False)
# vcf.rename(columns={'#CHROM':'CHROM'}, inplace=True)
IIDs = [x.split('/')[-1] for x in gtc_list]
vcf_melt = vcf.melt(id_vars=['CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO'], value_vars=IIDs, value_name='metrics')
vcf_melt[['GT','GQ','BAF','LRR']] = vcf_melt.metrics.str.split(':', expand=True)
vcf_melt.drop(columns=['QUAL','FILTER','INFO'], inplace=True)
vcf_melt.to_csv(f'{gtc_dir}/ibx_variant_metrics.txt', sep='\t', index=False)

In [5]:
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,204820990004_R01C01,...,204842400139_R07C01,204842400139_R08C01,204842400159_R01C01,204842400159_R02C01,204842400159_R03C01,204842400159_R04C01,204842400159_R05C01,204842400159_R06C01,204842400159_R07C01,204842400159_R08C01
0,1,49554,1:49554-G-A,A,G,.,PASS,.,GT:GQ:BAF:LRR,0/1:4:0.445278:0.267114,...,0/1:4:0.423465:0.0777876,0/0:4:0:0.491343,0/1:2:0.273933:0.807688,0/1:1:0.26082:0.768816,0/1:2:0.287269:0.668899,0/1:4:0.378495:0.377897,0/1:4:0.372401:0.254954,0/1:3:0.335069:0.391,./.:0:0:-3.64196,0/0:4:0:0.516008
1,1,115746,JHU_1.115745,C,T,.,PASS,.,GT:GQ:BAF:LRR,0/0:3:0.0949309:-0.00494799,...,0/0:3:0.0694953:-0.0622626,0/0:3:0.0659552:0.159177,0/0:3:0:0.334754,0/0:3:0:0.383493,0/0:3:0.000423431:0.393729,0/0:3:0:0.0198632,0/0:3:0.0255343:0.0354245,0/0:3:0:0.066139,0/0:3:0:0.0672,0/0:3:0:0.0664136
2,1,534238,JHU_1.534237,A,C,.,PASS,.,GT:GQ:BAF:LRR,0/0:2:0.00715753:-0.352969,...,0/0:2:0.0227675:-0.26038,0/0:2:0.0120197:-0.474795,0/0:2:0.00179036:-0.218394,0/0:2:0:-0.340575,0/0:2:0:-0.0732102,0/0:2:0:-0.320035,0/0:2:0.00316805:-0.372469,0/0:2:0:-0.358175,./.:0:0.00714883:-2.28197,0/0:2:0:-0.32114
3,1,564868,chr1:629488:T:C,T,C,.,PASS,.,GT:GQ:BAF:LRR,1/1:2:1:0.0232291,...,1/1:2:1:-0.0182266,1/1:2:1:0.315991,1/1:2:1:0.495458,1/1:2:1:0.439584,1/1:2:1:0.239748,1/1:2:1:-0.0356574,1/1:2:1:0.0953711,1/1:2:1:0.181234,./.:0:0.794614:1.9661,1/1:2:1:0.216032
4,1,565433,rs9701055,C,T,.,PASS,.,GT:GQ:BAF:LRR,0/0:3:0.0356333:-0.0100339,...,0/0:3:0:-0.2145,0/0:2:0.056486:-0.0766128,0/0:3:0:0.374188,0/0:3:0:0.4879,0/0:3:0:0.376024,0/0:3:0:-0.0227302,0/0:3:0:0.0424756,0/0:3:0:-0.153805,./.:0:0.0356801:1.25242,0/0:3:0.0185059:0.0548676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860385,MT,16443,rs371960162,T,C,.,PASS,.,GT:GQ:BAF:LRR,0:2:0:-0.0343127,...,0:2:0:-0.117647,0:2:0:0.298925,0:2:0:0.456442,0:2:0:0.617104,0:2:0:0.467374,0:2:0:0.0359379,0:2:0:0.124423,0:2:0:0.145154,.:0:0:3.95953,0:2:0:0.233909
1860386,MT,16456,rs142662828,G,A,.,PASS,.,GT:GQ:BAF:LRR,0:1:0.106751:-0.0626228,...,0:1:0.0471455:-0.105054,0:1:0.107425:0.0628178,0:1:0.0791517:0.372027,0:1:0.0426173:0.338313,0:1:0.0712738:0.222506,0:1:0.0798931:-0.00806458,0:1:0.0833367:-0.0806504,0:1:0.0719105:-0.0021004,.:0:0.704323:2.01215,0:1:0.124073:0.00706766
1860387,MT,16465,"chrM_16465,chrM_16465_ilmndup1",C,T,.,PASS,.,GT:GQ:BAF:LRR,0:3:0.0163963:0.145347,...,0:3:0.00738358:0.0221264,0:3:0.00386667:0.396264,0:3:0:0.585258,0:3:0:0.592106,0:3:0.00299925:0.513029,0:3:0:0.164402,0:3:0.00363982:0.178511,0:3:0.00350219:0.174209,.:0:0.0817734:2.23837,0:3:0:0.354196
1860388,MT,16482,200610-37,A,G,.,PASS,.,GT:GQ:BAF:LRR,0:4:0.0411672:-0.215295,...,0:4:0.00808741:-0.194064,0:4:0.0226893:0.090447,0:4:0:0.459284,0:4:0:0.548924,0:4:0:0.458283,0:4:0:0.0495727,0:4:0:0.000396881,0:4:0:0.0444468,.:0:0.0282701:2.21461,0:4:0:-0.0895109


In [85]:
import pandas as pd
import os
import shutil
import zipfile

from QC.qc import callrate_prune, het_prune, sex_prune, related_prune, variant_prune, avg_miss_rates
from Ancestry.ancestry import run_ancestry, split_cohort_ancestry
from QC.utils import shell_do



In [86]:
geno_path = f'{bed_dir}/ibx'


In [6]:
steps = []

avg_miss = avg_miss_rates(geno_path, f'{geno_path}_missing')
avg_miss

callrate_out = f'{geno_path}_callrate'
callrate = callrate_prune(geno_path, callrate_out, mind=0.03)
steps.append(callrate)

sex_out = f'{callrate_out}_sex'
sex = sex_prune(callrate_out, sex_out)
steps.append(sex)


RUNNING: callrate_prune



Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx --mind 0.03 --make-bed --out /data/vitaled2/ibx_data/bed/ibx_callrate



RUNNING: sex_prune



Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --check-sex 0.25 0.75 --maf 0.05 --out /data/vitaled2/ibx_data/bed/ibx_callrate_sex_tmp1
Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --chr 23 --from-bp 2699520 --to-bp 154931043 --maf 0.05 --geno 0.05 --hwe 1E-5 --check-sex 0.25 0.75 --out /data/vitaled2/ibx_data/bed/ibx_callrate_sex_tmp2
Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --remove /data/vitaled2/ibx_data/bed/ibx_callrate_sex.fails --make-bed --out /data/vitaled2/ibx_data/bed/ibx_callrate_sex


In [4]:
1-0.008408835276531221

0.9915911647234688

In [5]:
1-0.008410770833333333


0.9915892291666667

In [9]:
# related
related_out = f'{callrate_out}_related'
related = related_prune(callrate_out, related_out)



RUNNING: related_prune



Executing: gcta --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --autosome --maf 0.05 --make-grm --out /data/vitaled2/ibx_data/bed/ibx_callrate_related_total_grm_tmp
Executing: gcta --grm /data/vitaled2/ibx_data/bed/ibx_callrate_related_total_grm_tmp --grm-cutoff 0.125 --make-grm --out /data/vitaled2/ibx_data/bed/ibx_callrate_related_unrelated_grm_tmp
Executing: plink --bfile /data/vitaled2/ibx_data/bed/ibx_callrate --keep /data/vitaled2/ibx_data/bed/ibx_callrate_related_unrelated_grm_tmp.grm.id --make-bed --out /data/vitaled2/ibx_data/bed/ibx_callrate_related
Executing: gcta --grm /data/vitaled2/ibx_data/bed/ibx_callrate_related_total_grm_tmp --grm-cutoff 0.95 --make-grm --out /data/vitaled2/ibx_data/bed/ibx_callrate_related_duplicated_grm_tmp


{'pass': True,
 'step': 'related_prune',
 'metrics': {'related_count': 0, 'duplicated_count': 0},
 'output': {'pruned_samples': '/data/vitaled2/ibx_data/bed/ibx_callrate_related.related',
  'plink_out': '/data/vitaled2/ibx_data/bed/ibx_callrate_related'}}

In [103]:
idat_dir = f'{basedir}/ibx_plus_coriell'
ibx_only_samples = f'{basedir}/ped'


In [22]:
idat_to_ped_cmd = f'\
{iaap} gencall \
{bpm} \
{egt} \
{idat_dir}/ \
-f {idat_dir}/ \
-p \
-t 16'

shell_do(idat_to_ped_cmd)

Executing: /data/CARD/PD/GP2/ilmn_files/iaap-cli/iaap-cli gencall /data/CARD/PD/GP2/ilmn_files/NeuroBooster_20042459_A1.bpm /data/CARD/PD/GP2/ilmn_files/NBSCluster_file_n1393_011921.egt /data/CARD/PD/ibx_data/ibx_plus_coriell/ -f /data/CARD/PD/ibx_data/ibx_plus_coriell/ -p -t 16


In [87]:
ped_filenames = [p.split('.')[0] for p in glob.glob(f'{idat_dir}/*.ped')]
# map_file = f'{ped_dir}/NeuroBooster_20042459_A1.map'
# for filename in ped_filenames:
#     ped = f'{filename}.ped'
#     out_map = f'{filename}.map'
#     if os.path.isfile(ped):
#         shutil.copyfile(src=map_file, dst=out_map)
#     else:
#         print(f'{ped} does not exist!')
#         print(f'{out_map} creation cancelled')

In [90]:
ibx_samples = [y.split('.')[0] for y in [x.split('/')[-1] for x in glob.glob(f'{ibx_only_samples}/*.fam')]]


In [24]:
for filename in ped_filenames:
    
    ped = f'{filename}'
    make_bed_cmd = f'\
plink \
--file {ped} \
--make-bed \
--out {filename}'
    
    shell_do(make_bed_cmd)

Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204825020102_R07C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204825020102_R07C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204835450116_R07C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204835450116_R07C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204842400004_R06C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204842400004_R06C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204701860056_R08C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204701860056_R08C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204835450020_R01C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204835450020_R01C01
Executing: plink --file /data/CARD/PD/ibx_data/ibx_plus_coriell/204701860109_R06C01 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/204701860109_R06C01
Executing: plink --file /data/CARD

In [6]:
with open(f"{idat_dir}/merge_bed.list", 'w') as f:
    for filename in ped_filenames:
        bed = f'{filename}'
        f.write(f'{bed}\n')
f.close()

with open(f"{swarm_scripts_dir}/merge.swarm", 'w') as f:

    plink_merge_cmd = f'\
plink \
--merge-list {idat_dir}/merge_bed.list \
--make-bed \
--out {idat_dir}/ibx'
    f.write(f"{plink_merge_cmd}")
f.close()

In [8]:
!swarm -f {swarm_scripts_dir}/merge.swarm -g 64 -t 32 --time=10:00:00 --logdir swarm --gres=lscratch:20 --partition=norm

16856865


In [104]:
geno_path = f'{idat_dir}/ibx'
steps=[]

callrate_out = f'{geno_path}_callrate'
callrate = callrate_prune(geno_path, callrate_out, mind=0.03)
steps.append(callrate)

sex_out = f'{callrate_out}_sex'
sex = sex_prune(callrate_out, sex_out)
steps.append(sex)



RUNNING: callrate_prune



Executing: plink --bfile /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx --mind 0.03 --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate



RUNNING: sex_prune



Executing: plink --bfile /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate --check-sex 0.25 0.75 --maf 0.05 --out /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex_tmp1
Executing: plink --bfile /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate --chr 23 --from-bp 2699520 --to-bp 154931043 --maf 0.05 --geno 0.05 --hwe 1E-5 --check-sex 0.25 0.75 --out /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex_tmp2
Executing: plink --bfile /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate --remove /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex.fails --make-bed --out /data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex


In [108]:
# pruned_ibx_coriell = pd.read_csv('/data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex.fam',sep='\s+', names=['FID','IID','PAT','MAT','SEX','PHENO'])
# pruned_ibx_coriell[pruned_ibx_coriell.IID.isin(ibx_samples)]
callrate_outliers = pd.read_csv('/data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate.outliers',sep='\s+', header=None,names=['FID','IID'])
callrate_outliers[callrate_outliers.IID.isin(ibx_samples)]

,FID,IID
36,0,204842400139_R08C01


In [45]:
sexcheck = pd.read_csv('/data/CARD/PD/ibx_data/ibx_plus_coriell/ibx_callrate_sex_tmp1.sexcheck',sep='\s+')
sexcheck

,FID,IID,PEDSEX,SNPSEX,STATUS,F
0,0,204697840001_R01C01,2,2,OK,-0.005508
1,0,204697840001_R02C01,1,1,OK,0.999700
2,0,204697840001_R03C01,2,2,OK,0.065470
3,0,204697840001_R04C01,2,2,OK,0.167800
4,0,204697840001_R05C01,1,1,OK,0.999600
...,...,...,...,...,...,...
2347,0,204854950032_R04C01,1,1,OK,0.999700
2348,0,204854950032_R05C01,1,1,OK,0.999600
2349,0,204854950032_R06C01,1,1,OK,0.999700
2350,0,204854950032_R07C01,2,2,OK,0.028070


In [46]:
sexcheck[sexcheck.IID.isin(ibx_samples)].STATUS.value_counts()

OK    95
Name: STATUS, dtype: int64

In [47]:
sexcheck.STATUS.value_counts()

OK         2335
PROBLEM      17
Name: STATUS, dtype: int64

In [39]:
coriell_metrics = pd.read_hdf('/data/CARD/PD/GP2/genotypes/coriell/clean/coriell.QC.metrics.h5', key='QC')

In [14]:
# now merge previously-called genotypes (ibx + coriell) to see how pruning works

coriell_geno = '/data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell'
ibx_geno = f'{bed_dir}/ibx'
ibx_coriell_geno = f'{ibx_geno}_coriell'

plink_merge_cmd = f'plink \
--bfile {ibx_geno} \
--bmerge {coriell_geno} \
--allow-no-sex \
--make-bed \
--out {ibx_coriell_geno}'

shell_do(plink_merge_cmd)


Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx --bmerge /data/CARD/PD/GP2/raw_genotypes/coriell/plink/coriell --allow-no-sex --make-bed --out /data/CARD/PD/ibx_data/bed/ibx_coriell


In [16]:
geno_path = ibx_coriell_geno

In [17]:
steps = []

avg_miss = avg_miss_rates(geno_path, f'{geno_path}_missing')
avg_miss

callrate_out = f'{geno_path}_callrate'
callrate = callrate_prune(geno_path, callrate_out, mind=0.03)
steps.append(callrate)

sex_out = f'{callrate_out}_sex'
sex = sex_prune(callrate_out, sex_out)
steps.append(sex)

Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell --missing --out /data/CARD/PD/ibx_data/bed/ibx_coriell_missing


Average Missing Call Rate (lmiss): 0.01126679989325701
Average Missing Genotyping Rate (imiss): 0.011273141249999999

RUNNING: callrate_prune



Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell --mind 0.03 --make-bed --out /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate



RUNNING: sex_prune



Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate --check-sex 0.25 0.75 --maf 0.05 --out /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate_sex_tmp1
Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate --chr 23 --from-bp 2699520 --to-bp 154931043 --maf 0.05 --geno 0.05 --hwe 1E-5 --check-sex 0.25 0.75 --out /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate_sex_tmp2
Executing: plink --bfile /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate --remove /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate_sex.fails --make-bed --out /data/CARD/PD/ibx_data/bed/ibx_coriell_callrate_sex
